# Задача: предсказание исходов профессиональных матчей по cs:go
## Этап: извлечение признаков 
### Описание этапа:
На данном этапе извлекаются признаки для турнирных игр по cs:go. Каждая игра имеет 2 команды, которые могут начать за 2 стороны(террористы и контр-террористы), игры состоят из раундов и каждый раунд может закончиться одним из исходов(взрыв бомбы, обезвреживание бомбы, вышедшее время раунда, полное уничтожение команды соперников). Игры проводятся в рамках различных турниров, каждый из которых имеет свой призовой фонд(сумма+валюта) и тир; игры имеют различный уровень важности: групповая стадия, плей-офф. В игре участвуют 10 игроков, по каждому из которых имеется регистрационная информация(год рождения, национальность и др) и статистическая информация.

* Игровые статистики:

|название|описание|
|---|---|
|adr|средний урон за раунд|
|assists|число ассистов|
|deaths|число смертей
|first_kills_diff|
|flash_assists|число световых гранат, способствовавших убийству|
|headshots|число убийств в голову|
|k_d_diff|отношение числа убиств к числу смертей|
|kast|%раундов,в которых игрок имел убийства, ассисты, выжил или был разменен|
|kills|число убийств
|rating|hltv рейтинг|

* hltv rating
Прошлая версия рейтинга состояла из трех компонентов — среднего количества фрагов в раунде, среднего количества раундов без смертей и дополнительной оценка, основанной на количестве раундов с двумя и более фрагами.
В финальной версии рейтинг 2.0 учитывает стороны, за которые выступал игрок. Обновленный параметр Impact Rating теперь включает среднее количество раундов с 2+ фрагами, клатчи и т.д. Появились два новых компонента — KAST Rating и Damage Rating. Первый отражает процент раундов без смертей, с фрагом, ассистом или разменом; второй — средний нанесенный урон в раунде.

### Признаки:

* принцип построения признаков: для игры агрегируется статистика по каждой команде во всех играх, состоявшихся до текущей на той же игровой карте                   
* список признаков:  

#### признаки для команды

    1. длина истории игр команды                  
    2. частоты бинаризованных длительностей игр
    3. частоты лет, месяцев, дней, дней недели, часов, десятков минут                   
    4. частоты исходов каждого раунда за каждую сторону с учетом результата раунда(взрыв бомбы, обезвреживание бомбыб ...)
    5. частоты категорий (тир серии, национальность, ...)
    6. суммарный призовой фонд в долларах
    7. частоты стадий турнира (группа, плей-офф, ...)
    8. ранжированные боевые статистики игроков в командах в пересчете на 1 раунд
    9. ранжированные личные боевые статистики игроков в пересчете на 1 раунд
    10. п.8 и п.9 для игр в рамках года, месяца, дня, дня недели, часа   
    
#### суммарное число выигранных раундов каждой командой в разрезе карт и сторон начала во всех играх, закончившихся до начала текущей      

### Целевой признак:

примеры целевого признака:   
    1. контр-террористы выиграют больше 10 раундов, а террористы больше3    
    2. всего будет сыграно меньше 25 раундов
    3. ...
    
### Дальнейшие действия:
    1. отбор признаков
    2. формирование целевых признаков

In [1]:
import pandas as pd
import numpy as np
import json, pickle, os, gc, re, time
from tqdm import tqdm, tqdm_notebook
from datetime import datetime
import dateutil.parser
import warnings
warnings.filterwarnings('ignore')
from collections import defaultdict

class CsgoFeatureGeneratorForGameWithMap():
    def __init__(self, PATH_TO_GAMES_OUT):
        self.PATH_TO_GAMES_OUT = PATH_TO_GAMES_OUT
    def _json_iterator(self, path_to_directory):                        
        filenames = os.listdir(path_to_directory)
        for i, filename in tqdm(enumerate(filenames), total =len(filenames)):              
            pth = os.path.join(path_to_directory, filename)
            with open(pth, 'r') as f:
                current_game = json.load(f)
            yield current_game 
    def _build_df(self, PATH_TO_GAMES_OUT):
        # списки с трансформированными json в pd.DataFrame
        L_rows = []        
        for rsp in self._json_iterator(PATH_TO_GAMES_OUT):
            L_rows.append(pd.json_normalize(rsp))
        # финальный датафрейм
        df = pd.concat(L_rows)
        df['begin_at'] = df['begin_at'].apply(lambda x: pd.to_datetime(x).tz_localize(None))
        df['end_at'] = df['end_at'].apply(lambda x: pd.to_datetime(x).tz_localize(None))
        return df.reset_index(drop = True)
    def _get_game_history_4team_and_map(self, df, row, team1_id,map_id):    
        MASK_CurrentTeam_in_HistoryGamesTeams = df['teams'].apply(lambda x: team1_id in (x[0]['id'], x[1]['id']))
        MASK_CurrentMap_equals_HistoryGamesMap = df['map.id'] == map_id
        MASK_CurrentGame_after_HistoryGames = df['begin_at']<row['begin_at']
        MASK=MASK_CurrentTeam_in_HistoryGamesTeams&MASK_CurrentMap_equals_HistoryGamesMap & MASK_CurrentGame_after_HistoryGames
        df_history4team= df.drop(row.name, 0)[MASK]
        return df_history4team
    def _add_frequencies(self,d_features, y, name):
        d_counts = y.value_counts(normalize = True).to_dict()
        for k,value in d_counts.items():
            key = '{}.{}'.format(name, k)
            d_features[key] = value

    def _agg_global_info(self,df_history_4team1, team1_id, map_id):

        d_features = {}
        d_features['map.id'] = map_id
        d_features['game_history.count'] = df_history_4team1.shape[0]
        games_duration_minute = ((df_history_4team1['end_at'] - df_history_4team1['begin_at']) // np.timedelta64(60, 's'))

        begin_at = df_history_4team1['begin_at']
        y = begin_at.dt.year.astype(int)
        m = begin_at.dt.month.astype(int)
        d = begin_at.dt.day.astype(int)
        dow = begin_at.dt.dayofweek.astype(int)
        h = begin_at.dt.hour.astype(int)
        mnt = (begin_at.dt.minute/10).round().astype(int)
        begin_at_keys = ('begin_at.year','begin_at.month','begin_at.day',
                         'begin_at.dayofweek', 'begin_at.hour', 'begin_at.minute(//10)')
        bins = np.arange(1, 121)
        for n_minutes in bins:
            key = 'game_duration.<={}(minutes).share'.format(n_minutes)
            d_features[key] = (games_duration_minute >= n_minutes).mean()

        begin_at_values = (y, m, d, dow, h, mnt)
        for name, vals in zip(begin_at_keys,begin_at_values):
            self._add_frequencies(d_features, vals, name = name)

        df_history_4team1.columns
        for key in ['match.league_id', 'match.serie_id', 'match.tournament_id',\
                    'match.serie.tier', 'match.tournament.name']:
            vals = df_history_4team1[key]
            self._add_frequencies(d_features, vals, name = key)

        trnmnt_pp = df_history_4team1['match.tournament.prizepool'] 
        dollars_mask = trnmnt_pp.apply(lambda x: ''.join(x.split(' ')[1:]) == 'UnitedStatesDollar')        
        total_pp = trnmnt_pp[dollars_mask].apply(lambda x: int(''.join(re.findall(r'\d+', x.split(' ')[0])))).sum()
        d_features['tournament.prizepool.sum'] = total_pp

        return d_features
    def _agg_rounds(self,df_history_4team1, team1_id):
        DD_rounds = defaultdict(list)
        for game in df_history_4team1['rounds']:
            
            df_rounds= pd.DataFrame.from_records(game)
            r_min= df_rounds['round'].min()
            if r_min <=15:
                subdf = df_rounds[df_rounds['round']== r_min]
                start_ct = subdf['ct'].iloc[0]
                start_t = subdf['terrorists'].iloc[0]
            else:
                subdf = df_rounds[df_rounds['round']== r_min]
                start_ct = subdf['terrorists'].iloc[0]
                start_t = subdf['ct'].iloc[0]

            MASK_team1_start_ct = start_ct == team1_id
            MASK_team1_start_t = start_ct != team1_id   

            for r in game:
                r_id = int(r['round'])
                winner_id = int(r['winner_team'])

                MASK_team1_win = winner_id == team1_id
                MASK_team1_lose = winner_id != team1_id

                DD_rounds['start_ct.r{}.{}.win'.format(r_id, r['outcome'])].append(MASK_team1_start_ct & MASK_team1_win)
                DD_rounds['start_t.r{}.{}.win'.format(r_id, r['outcome'])].append(MASK_team1_start_t & MASK_team1_win)
                DD_rounds['start_ct.r{}.{}.lose'.format(r_id, r['outcome'])].append(MASK_team1_start_ct & MASK_team1_lose)
                DD_rounds['start_t.r{}.{}.lose'.format(r_id, r['outcome'])].append(MASK_team1_start_t & MASK_team1_lose)   

            D_rounds = {k:np.mean(v) for k, v in DD_rounds.items()}
            return D_rounds
            
        
    def _agg_stats(self,df_history_4team1, team1_id):
        DD_players_personal = defaultdict(list)
        DD_players_ranked = defaultdict(list)
        L_player_reginfo = []
        L_players_used = []

        L1_stat_keys = ['assists', 'deaths',  'first_kills_diff', 'flash_assists',
                        'headshots', 'kills']
        L2_stat_keys = ['adr', 'k_d_diff', 'kast', 'rating']
        for i, L_players in enumerate(df_history_4team1['players']):
            game_total_rounds = df_history_4team1['rounds'].iloc[i][-1]['round']
            for player in L_players:
                player_id = int(player['player']['id'])
                player_teamid = int(player['team']['id'])
                if player_teamid == team1_id:
                    bd =pd.to_datetime(player['player']['birthday'])
                    if player_id not in L_players_used:
                        L_player_reginfo.append({'bd_year':bd.year, 'bd.month':bd.month,\
                                                 'bd_day':bd.day, 'hometown':player['player']['hometown'],\
                                                 'nationality':player['player']['nationality']})
                        L_players_used.append(player_id)

                    for key_stat in L1_stat_keys:
                        DD_players_personal['player{}.{}.mean_per_round'\
                                            .format(player_id,key_stat)]\
                                            .append(player[key_stat]/game_total_rounds)
                        DD_players_ranked['player{}.{}.mean_per_round'\
                                              .format(i, key_stat)]\
                                              .append(player[key_stat]/game_total_rounds)
                    for key_stat in L2_stat_keys:
                        DD_players_personal['player{}.{}.mean'\
                                            .format(player_id,key_stat)]\
                                            .append(player[key_stat])
                        DD_players_ranked['player{}.{}.mean'\
                                          .format(i, key_stat)]\
                                          .append(player[key_stat])

        D_players_personal = {k:np.mean(v) for k,v in DD_players_personal.items()}

        DD_players_ranked = {k:sorted(v) for k,v in DD_players_ranked.items()} 
        DD2 = defaultdict(list)
        for k, L_ranked_values in DD_players_ranked.items():
            for rank, value in enumerate(L_ranked_values):
                new_key = '{}.rank{}'.format('.'.join(k.split('.')[1:]), rank+1)            
                DD2[new_key].append(value) 
        D_players_ranked = {k:np.mean(v) for k, v in DD2.items()}  

        DD_player_reginfo = defaultdict(list)
        for d in L_player_reginfo:
            for  k, v in d.items():            
                DD_player_reginfo[k].append(v)
        D_player_reginfo = {k:pd.Series(v).value_counts(normalize = True) for k, v in DD_player_reginfo.items()}


        return D_player_reginfo, D_players_personal, D_players_ranked   
    
    def reduce_mem_usage(self, df):
        numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
        for col in df.columns:
            col_type = df[col].dtypes
            if col_type in numerics:
                c_min = df[col].min()
                c_max = df[col].max()
                if str(col_type)[:3] == 'int':
                    if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                        df[col] = df[col].astype(np.int8)
                    elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                        df[col] = df[col].astype(np.int16)
                    elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                        df[col] = df[col].astype(np.int32)
                    elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                        df[col] = df[col].astype(np.int64)
                else:
                    if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                        df[col] = df[col].astype(np.float16)
                    elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                        df[col] = df[col].astype(np.float32)
                    else:
                        df[col] = df[col].astype(np.float64)

        return df

    def fit(self, PATH_TO_FEATURES, PATH_TO_TARGETS):        
        
        for _dir in [PATH_TO_FEATURES, PATH_TO_TARGETS]:
            if not(os.path.isdir(_dir)):
                os.mkdir(_dir)
            else:
                raise ValueError('directory already exists ...')
        
        lstdr = os.listdir(PATH_TO_FEATURES)
        
        # из имеющихся логов собираем датафрейм
        df = self._build_df(self.PATH_TO_GAMES_OUT) 
        
        # список с признаками
        self.L_all_features = []
        
        # список с исходами раундов в текущей игре
        self.L_rounds = []
            
        # проходим по играм
        for i in tqdm(range(len(df))):
            
            _fn = '{}.pickle'.format(i)
            if _fn not in lstdr: 
                # текущая игра
                row= df.iloc[i]


                # команда1, команда2, карта
                team1_id, team2_id, map_id = [int(t['id']) for t in row['teams']] + [int(row['map.id'])]

                # список с признаками
                L_features = []

                # для каждой команды
                for j, current_team_id in enumerate((team1_id, team2_id)):

                    # собираем игры на карте из текщей игры
                    # в которых участовала текущая команда
                    # завершившиеся до текущей игры
                    df_history_4team1 = self._get_game_history_4team_and_map(df, row, current_team_id,map_id)                

                    # раунды
                    d_agg_rounds = self._agg_rounds(df_history_4team1, current_team_id)
                    if d_agg_rounds is not None:
                        break_mask = False
                    else:
                        break_mask = True
                    if break_mask == False:

                        # глобальные статистики
                        d_features = self._agg_global_info(df_history_4team1, current_team_id, map_id)
                        d_features['team_id'] = current_team_id
                        # команды, игроки
                        d_player_reginfo, D_players_personal, D_players_ranked = self._agg_stats(df_history_4team1, \
                                                                                                 current_team_id)
                        for k, d_values in d_player_reginfo.items():
                            for k2, value in d_values.items():          
                                new_key = '{}.{}'.format(k, k2)
                                d_features[new_key] = value
                        d_features.update(D_players_personal)
                        d_features.update(D_players_ranked)

                        # команды в разрезе года, месяца, дня, дня недели, часа
                        years = df_history_4team1['begin_at'].dt.year
                        months = df_history_4team1['begin_at'].dt.month
                        days = df_history_4team1['begin_at'].dt.day
                        dayofweeks = df_history_4team1['begin_at'].dt.dayofweek
                        hours = df_history_4team1['begin_at'].dt.round('H').dt.hour
                        L_series = [years, months, days, dayofweeks, hours]
                        L_names = ['year', 'month', 'day', 'dayofweek', 'hour']
                        for name, series in zip(L_names, L_series):
                            unique_values = series.unique()
                            for unique_value in unique_values:
                                subdf = df_history_4team1[series == unique_value]
                                d_ranked_stats_4unique_value = self._agg_stats(subdf, current_team_id)[2]
                                d_ranked_stats_4unique_value_c = {}
                                for k, v in d_ranked_stats_4unique_value.items():
                                    new_k = '{}.{}.{}'.format(k, name, unique_value)
                                    d_ranked_stats_4unique_value_c[new_k] = v                            
                                d_features.update(d_ranked_stats_4unique_value_c)                

                        # строка с признаками
                        L_features.append(pd.DataFrame.from_records([d_features]).add_prefix('TEAM{}.'.format(j+1)))

                    else:
                        pass
            
            try:
                # пополняем признаки
                d_row_features = pd.concat(L_features, 1).iloc[0].to_dict()                
                pth = os.path.join(PATH_TO_FEATURES, '{}.pickle'.format(i))
                with open(pth, 'wb') as f:
                    pickle.dump(d_row_features, f)
                                    
                # пополняем исходы раундов
                L_rounds = row['rounds']
                pth = os.path.join(PATH_TO_TARGETS, '{}.pickle'.format(i))
                with open(pth, 'wb') as f:
                    pickle.dump(L_rounds, f)                
            except:
                pass                
            
        return self
    
    
class CsgoFeatureGeneratorForTeamWithSide():
    def __init__(self, PATH_TO_GAMES_OUT):
        self.PATH_TO_GAMES_OUT = PATH_TO_GAMES_OUT
    def _json_iterator(self, path_to_directory):                        
        filenames = os.listdir(path_to_directory)
        for i, filename in tqdm(enumerate(filenames), total =len(filenames)):              
            pth = os.path.join(path_to_directory, filename)
            with open(pth, 'r') as f:
                current_game = json.load(f)
            yield current_game 
    def _build_df(self, PATH_TO_GAMES_OUT):
        # списки с трансформированными json в pd.DataFrame
        L_rows = []        
        for rsp in self._json_iterator(PATH_TO_GAMES_OUT):
            L_rows.append(pd.json_normalize(rsp))
        # финальный датафрейм
        df = pd.concat(L_rows)
        df['begin_at'] = df['begin_at'].apply(lambda x: pd.to_datetime(x).tz_localize(None))
        df['end_at'] = df['end_at'].apply(lambda x: pd.to_datetime(x).tz_localize(None))
        return df.reset_index(drop = True)
    def _agg_sum_final_scores_by_side(self, df_history):
        d_scores_for_startside_and_map = {}
        for j in range(len(df_history)):
            row2 = df_history.iloc[j]
            map_id = int(row2['map.id'])

            df_rounds = pd.DataFrame.from_records(row2['rounds'])
            r_min = df_rounds[df_rounds['round'] == df_rounds['round'].min()].iloc[0]
            if r_min['round']<=15:
                d_sides = {r_min['ct']:'ct', r_min['terrorists']:'t'}
            else:
                d_sides = {r_min['ct']:'t', r_min['terrorists']:'ct'}
            for d_score in row2['rounds_score']:
                new_key = 'teamid{}.mapid{}.side_{}'.format(int(d_score['team_id']), map_id, d_sides[d_score['team_id']])    
                try:
                    d_scores_for_startside_and_map[new_key]+=d_score['score']
                except:
                    d_scores_for_startside_and_map[new_key]=d_score['score']            
        return d_scores_for_startside_and_map
    
    def reduce_mem_usage(self, df):
        numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
        for col in df.columns:
            col_type = df[col].dtypes
            if col_type in numerics:
                c_min = df[col].min()
                c_max = df[col].max()
                if str(col_type)[:3] == 'int':
                    if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                        df[col] = df[col].astype(np.int8)
                    elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                        df[col] = df[col].astype(np.int16)
                    elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                        df[col] = df[col].astype(np.int32)
                    elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                        df[col] = df[col].astype(np.int64)
                else:
                    if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                        df[col] = df[col].astype(np.float16)
                    elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                        df[col] = df[col].astype(np.float32)
                    else:
                        df[col] = df[col].astype(np.float64)

        return df
    
    def fit(self, PATH_TO_FEATURES_V2):
        if not(os.path.isdir(PATH_TO_FEATURES_V2)):
            os.mkdir(PATH_TO_FEATURES_V2)
        else:
            raise ValueError('directory already exists ...')
        df = self._build_df(self.PATH_TO_GAMES_OUT)
        for i in tqdm(range(len(df))):
            row = df.iloc[i]
            df_history = df[df['begin_at'] < row['begin_at']]
            try:
                d_scores_for_startside_and_map = self._agg_sum_final_scores_by_side(df_history)
                pth = os.path.join(PATH_TO_FEATURES_V2, '{}.pickle'.format(i))
                with open(pth, 'wb') as f:
                    pickle.dump(d_scores_for_startside_and_map, f)  
            except:
                pass
        return self        

In [2]:
PATH_TO_GAMES_OUT = r'./games_out'
PATH_TO_FEATURES_OUT = r'./features_for_2teams_and_map'
PATH_TO_TARGETS_OUT = r'./targets'
PATH_TO_FEATURES_V2_OUT = r'./features_for_startside_and_map'

In [ ]:
csgo_feature_gen = CsgoFeatureGeneratorForGameWithMap(PATH_TO_GAMES_OUT)
csgo_feature_gen.fit(PATH_TO_FEATURES_OUT, PATH_TO_TARGETS_OUT)

  2%|▏         | 586/27857 [06:30<10:07:04,  1.34s/it]

In [ ]:
csgo_feature_gen_v2 = CsgoFeatureGeneratorForTeamWithSide(PATH_TO_GAMES_OUT)
csgo_feature_gen_v2.fit(PATH_TO_FEATURES_V2_OUT)